# Exporting Hyperonym Barba

This Colab notebook contains instructions on how to export a servable Hyperonym Barba model that's ready for end-to-end serving and containerized deployment.

## Mount Google Drive

Mount Google Drive to the local file system:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Change working directory into Google Drive:

In [ ]:
%mkdir -p /content/drive/MyDrive/hyperonym/barba
%cd /content/drive/MyDrive/hyperonym/barba

/content/drive/MyDrive/hyperonym/barba


## Install dependencies

Install TensorFlow and related libraries:

In [ ]:
!pip install tensorflow==2.11.0 tensorflow-text==2.11.0 tensorflow-serving-api==2.11.0

In [ ]:
import tensorflow as tf
import tensorflow_text as tf_text

## Prepare components

In [ ]:
%mkdir -p models

### Load SentencePiece tokenizer

Download tokenizer model from Hugging Face if no local copy is available:

In [ ]:
!wget -nc -O models/sentencepiece.bpe.model "https://huggingface.co/xlm-roberta-base/resolve/main/sentencepiece.bpe.model"

Create SentencePiece tokenizer from the model:

In [ ]:
with open('models/sentencepiece.bpe.model', mode='rb') as file:
  tokenizer_model = file.read()

In [ ]:
tokenizer = tf_text.SentencepieceTokenizer(
  model=tokenizer_model,
  out_type=tf.int32,
  nbest_size=0,
  alpha=1.0,
  reverse=False,
  add_bos=False,
  add_eos=False,
  return_nbest=False,
  name=None
)

### Load fine-tuned model

Load the fine-tuned model from Google Drive:

In [ ]:
model = tf.saved_model.load('models/barba')

## Build servable model

In [ ]:
class Servable(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.tokenizer = tokenizer
    self.model = model

  def tokenize(self, inputs):
    ids = self.tokenizer.tokenize(inputs) + 1
    ids = tf.where(ids == 1, 3, ids) # <unk> = 0 -> 3
    return ids

  def concat(self, hypothesis_ids, premise_ids):
    n = hypothesis_ids.bounding_shape()[0]
    bos = tf.zeros([n, 1], dtype=hypothesis_ids.dtype) # <s> = 0
    eos = tf.ones([n, 1], dtype=hypothesis_ids.dtype) * 2 # </s> = 2
    ids = tf.concat([bos, hypothesis_ids, eos, eos, premise_ids, eos], axis=-1)
    return ids.to_tensor(default_value=1) # <pad> = 1

  def infer_ids(self, ids):
    mask = tf.cast(ids != 1, tf.int32) # <pad> = 1
    fn = self.model.signatures['serving_default']
    logits = fn(input_ids=ids, attention_mask=mask)['logits']
    return tf.nn.softmax(logits)[:,0]

  def infer_text(self, hypothesis, premise):
    hypothesis_ids = self.tokenize(hypothesis)
    premise_ids = self.tokenize(premise)
    ids = self.concat(hypothesis_ids, premise_ids)
    return self.infer_ids(ids)

  def call(self, inputs):
    return self.infer_text(inputs[0], inputs[1])

  def cartesian(self, inputs):
    n = tf.shape(inputs[0])[0]
    hypothesis = tf.tile(inputs[0], tf.shape(inputs[1]))
    premise = tf.repeat(inputs[1], n)
    probs = self.infer_text(hypothesis, premise)
    return tf.reshape(probs, [-1, n])

In [ ]:
servable = Servable()

### Test default signature

In [ ]:
hypotheses = tf.constant([
  'The church has cracks in the ceiling.',
  '美国宪法的制定有富兰克林参与'
], dtype=tf.string)
premises = tf.constant([
  'This church choir sings to the masses as they sing joyous songs from the book at a church.',
  '富兰克林不仅是美国大陆会议的代表及《独立宣言》起草和签署人之一，还是美国制宪会议代表及《美利坚合众国宪法》签署人之一。'
], dtype=tf.string)

In [ ]:
servable([hypotheses, premises])

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.00123494, 0.88700914], dtype=float32)>

### Test cartesian signature

In [ ]:
hypotheses_cartesian = tf.constant([
  'foreign policy',
  'Europe',
  'elections',
  'business',
  'outdoor recreation',
  'politics'
], dtype=tf.string)
premises_cartesian = tf.constant([
  'Who are you voting for in 2020?',
  '北欧有很不错的滑雪场'
], dtype=tf.string)

In [ ]:
servable.cartesian([hypotheses_cartesian, premises_cartesian])

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[0.01688034, 0.08493372, 0.25123692, 0.02789526, 0.04771834,
        0.24556275],
       [0.05800773, 0.5039153 , 0.01677088, 0.07735896, 0.83564895,
        0.06995673]], dtype=float32)>

## Export servable model

In [ ]:
%mkdir -p servables

### Save model with signatures

In [ ]:
signatures = dict()

Define the `serving_default` signature:

In [ ]:
@tf.function(input_signature=[
  [
    tf.TensorSpec([None], tf.string, 'hypothesis'),
    tf.TensorSpec([None], tf.string, 'premise')
  ]
])
def serving_default(inputs):
  outputs = dict()
  outputs['entailment'] = servable(inputs)
  return outputs

In [ ]:
signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = serving_default

Define the `cartesian` signature:

In [ ]:
@tf.function(input_signature=[
  [
    tf.TensorSpec([None], tf.string, 'hypothesis'),
    tf.TensorSpec([None], tf.string, 'premise')
  ]
])
def cartesian(inputs):
  outputs = dict()
  outputs['entailment'] = servable.cartesian(inputs)
  return outputs

In [ ]:
signatures['cartesian'] = cartesian

Save the servable model to Google Drive:

In [ ]:
save_dir = 'servables/barba/1'
tf.saved_model.save(servable, save_dir, signatures=signatures)

Create `assets.extra` directory:

In [ ]:
import os
extra_dir = os.path.join(save_dir, 'assets.extra')
os.makedirs(extra_dir, exist_ok=True)

### Embed warmup requests

Prepare warmup requests:

In [ ]:
hypotheses_warmup = tf.constant([
  'A robot should protect human',
  '机器人应当保护人类'
], dtype=tf.string)
premises_warmup = tf.constant([
  'A robot may not injure a human being or, through inaction, allow a human being to come to harm',
  '机器人不得伤害人类，或坐视人类受到伤害'
], dtype=tf.string)

Write warmup requests to file:

In [ ]:
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_log_pb2
warmup_record_path = os.path.join(extra_dir, 'tf_serving_warmup_requests')
with tf.io.TFRecordWriter(warmup_record_path) as writer:
  request = predict_pb2.PredictRequest()
  signature_name = tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY
  request.model_spec.name = tf.saved_model.SERVING
  request.model_spec.signature_name = signature_name
  request.inputs['hypothesis'].CopyFrom(tf.make_tensor_proto(hypotheses_warmup))
  request.inputs['premise'].CopyFrom(tf.make_tensor_proto(premises_warmup))
  predict_log = prediction_log_pb2.PredictLog(request=request)
  log = prediction_log_pb2.PredictionLog(predict_log=predict_log)
  writer.write(log.SerializeToString())

### Embed batching parameters

In [ ]:
batching_parameters_path = os.path.join(extra_dir, 'batching_parameters.pbtxt')
with open(batching_parameters_path, 'w') as file:
  file.write('max_batch_size { value: 32 }\n')
  file.write('batch_timeout_micros { value: 2000 }\n')
  file.write('num_batch_threads { value: 4 }\n')
  file.write('max_enqueued_batches { value: 100 }\n')

Flush and unmount Google Drive:

In [ ]:
drive.flush_and_unmount()